In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from splinter import Browser
import time
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
url = 'https://www.usich.gov/tools-for-action/map/#fn[]=1400&fn[]=2900&fn[]=6000&fn[]=9900&fn[]=13500'

# using the .visit() method tell the browser to visit the url
browser.visit(url)

#sleep code to allow browser page to load
time.sleep(5)

#obtain the page html using the .html attribute of the browser object
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [4]:
#Note the find method gets the "ul" tag with id = "stats", corresponding to homeless stats sections of the html
stats_tag =  soup.find("ul", id="stats")

""" Create a beautiful soup object for the <ul> tag that has id = "selectMenuBox"
Note this object also refers to the dropdown menu which contains a list of states with links to different pages
with data pertaining to the particular state """

state_menu_tag =  soup.find("ul", id="selectMenuBox")

#Create a list for all the <li> tag contained within the <ul> tag
states_tag_list = state_menu_tag.find_all("li")

#Based on the list of <li> tags create a list of the 'href's with link to the approriate state pages
href_list = [li.a['href'] for li in states_tag_list]
    
states = [i.a.text for i in states_tag_list]

In [5]:
#create a list to hold the dictionaries we will use to construct a pandas dataframe.
homeless_data = []

for x in range(len(states)):
    
    #Create an empty dictionary which we will use to create key-value pairs using the <span> text and the <p> text
    stats_dict = {} 
    
    #Click on the dropdown menu by referencing the <ul> id
    browser.click_link_by_id('selectBox')
    time.sleep(1)
    
    #use method to travel to next state page in list 
    browser.click_link_by_partial_href(href_list[x])
    time.sleep(5)
    
    #obtain the new page html using the .html attribute of the browser object
    state_html = browser.html

    # Create BeautifulSoup object; parse with 'html.parser'
    state_soup = BeautifulSoup(state_html, 'html.parser')
    
    #Note the find method gets the "ul" tag with id = "stats", corresponding to homeless stats sections of the html
    state_stats_tag =  state_soup.find("ul", id="stats")
    
    # create a list of <li>'s found in the stats <ul>
    state_stats_list = state_stats_tag.find_all("li")
    
    for i in state_stats_list:
        stats_dict[i.span.text] = i.p.text
    homeless_data.append(stats_dict)

In [9]:
homeless_df = pd.DataFrame(homeless_data, index=states)
homeless_df

Nighttime Residence: Doubled up  \
Alabama                                       12,021   
Alaska                                         2,257   
Arizona                                       15,049   
Arkansas                                      10,178   
California                                   211,607   
Colorado                                      17,146   
Connecticut                                    2,702   
Delaware                                       2,449   
District of Columbia                               0   
Florida                                       53,582   
Georgia                                       27,543   
Guam                                               0   
Hawaii                                         2,108   
Idaho                                          5,921   
Illinois                                      42,223   
Indiana                                       13,872   
Iowa                                           4,753   
Kansas                                         7,646   
Kentucky                                      21,328   
Louisiana                                     17,122   
Maine                                          1,383   
Maryland                                      12,810   
Massachusetts                                  9,780   
Michigan                                      27,923   
Minnesota                                      9,526   
Mississippi                                    8,630   
Missouri                                      26,707   
Montana                                        2,141   
Nebraska                                       2,023   
Nevada                                        15,150   
New Hampshire                                  2,499   
New Jersey                                     8,540   
New Mexico                                     7,930   
New York                                      88,580   
North Carolina                                19,131   
North Dakota                                   1,440   
Ohio                                          22,533   
Oklahoma                                      21,958   
Oregon                                        17,354   
Pennsylvania                                  14,362   
Puerto Rico                                    2,386   
Rhode Island                                     672   
South Carolina                                 9,423   
South Dakota                                   1,113   
Tennessee                                     11,154   
Texas                                         91,348   
Utah                                          13,005   
Vermont                                          670   
Virgin Islands                                     0   
Virginia                                      13,425   
Washington                                    28,582   
West Virginia                                  6,896   
Wisconsin                                     14,338   
Wyoming                                          920   

                     Nighttime Residence: Hotels/motels  \
Alabama                                             681   
Alaska                                              270   
Arizona                                           1,404   
Arkansas                                            524   
California                                       10,095   
Colorado                                          2,205   
Connecticut                                         396   
Delaware                                            561   
District of Columbia                                  0   
Florida                                           8,113   
Georgia                                           6,700   
Guam                                                  0   
Hawaii                                                5   
Idaho                                               413   
Illinois                                          1,795

In [55]:
df_columns = list(homeless_df.columns)

In [41]:
for x in df_columns:
    for i in range(len(homeless_df[x])):
        homeless_df[x][i] = homeless_df[x][i].replace(',', '')

for t in df_columns:
    homeless_df[t] = pd.to_numeric(homeless_df[t])

In [56]:
homeless_num = homeless_df['Total Homeless Population'].sum()

percentage = [((i/homeless_num)*100) for i in homeless_df['Total Homeless Population']]

homeless_df['Percentage of Homeless Population'] = percentage

homeless_df



Nighttime Residence: Doubled up  \
Alabama                                         12021   
Alaska                                           2257   
Arizona                                         15049   
Arkansas                                        10178   
California                                     211607   
Colorado                                        17146   
Connecticut                                      2702   
Delaware                                         2449   
District of Columbia                                0   
Florida                                         53582   
Georgia                                         27543   
Guam                                                0   
Hawaii                                           2108   
Idaho                                            5921   
Illinois                                        42223   
Indiana                                         13872   
Iowa                                             4753   
Kansas                                           7646   
Kentucky                                        21328   
Louisiana                                       17122   
Maine                                            1383   
Maryland                                        12810   
Massachusetts                                    9780   
Michigan                                        27923   
Minnesota                                        9526   
Mississippi                                      8630   
Missouri                                        26707   
Montana                                          2141   
Nebraska                                         2023   
Nevada                                          15150   
New Hampshire                                    2499   
New Jersey                                       8540   
New Mexico                                       7930   
New York                                        88580   
North Carolina                                  19131   
North Dakota                                     1440   
Ohio                                            22533   
Oklahoma                                        21958   
Oregon                                          17354   
Pennsylvania                                    14362   
Puerto Rico                                      2386   
Rhode Island                                      672   
South Carolina                                   9423   
South Dakota                                     1113   
Tennessee                                       11154   
Texas                                           91348   
Utah                                            13005   
Vermont                                           670   
Virgin Islands                                      0   
Virginia                                        13425   
Washington                                      28582   
West Virginia                                    6896   
Wisconsin                                       14338   
Wyoming                                           920   

                      Nighttime Residence: Hotels/motels  \
Alabama                                              681   
Alaska                                               270   
Arizona                                             1404   
Arkansas                                             524   
California                                         10095   
Colorado                                            2205   
Connecticut                                          396   
Delaware                                             561   
District of Columbia                                   0   
Florida                                             8113   
Georgia                                             6700   
Guam                                                   0   
Hawaii                                                 5   
Idaho                                         

In [58]:
homeless_df.to_csv("dataframes/2018_homeless_data.csv")